# Create EDD Study Files From Data

In [ ]:
cd ../

In [1]:
import pandas as pd
from IPython.display import display

In [2]:
df = pd.read_csv('data/indigoidine/indigoidine_media_Cycle1.csv')
df = df.loc[df['Plate']=='dest_plate']
#df = df.set_index(['Plate','Well'])

In [3]:
#Create Line Description
def create_media_description(series):
    description = ''
    for item,value in series.iteritems():
        if item not in ['Plate','Well','Target','Volume']:
            description += '{}:{:0.4f},'.format(item,value)
    return description
    
df['Line Description'] = df.apply(create_media_description,axis=1)

In [4]:
#Add Metadata for Media and Replicates To Craft Line Names
reps=3
n = 15

#flat_list = [item for sublist in  for item in sublist]

lnfcn = lambda x: 'M{}-R{}-W{}-C1-B1'.format(x['media'],x['replicate'],x['Well'])
df['media'] =     [i+1 for i in range(n) for _ in range(reps)]
df['replicate'] = [i+1 for _ in range(n) for i in range(reps)]
df['Line Name'] = df.apply(lnfcn,axis=1)

In [5]:
#Augment With Control Rows
data = {'Line Name':['M0-R1-W32-C1-B1','M0-R2-W40-C1-B1','M0-R3-W48-C1-B1'],'Line Description':['control','control','control'],'Well':[32,40,48]}
test_df = pd.DataFrame(data)
display(test_df)


,Line Name,Line Description,Well
0,M0-R1-W32-C1-B1,control,32
1,M0-R2-W40-C1-B1,control,40
2,M0-R3-W48-C1-B1,control,48


In [6]:
#Append Data To Data Frame
production_df = pd.read_csv('data/indigoidine/cycle1_batch1_data.csv')
display(production_df)

,Indigoidine [g/L]
0,0.039563
1,0.073779
2,0.082351
3,0.149748
4,0.204138
5,0.217884
6,-0.018005
7,0.040154
8,-0.037958
9,0.130090


In [7]:
df = pd.concat([df,test_df],sort=False)
df['Media'] = 'LB'
df['Part ID'] = ''
df['CID:193349:Indigoidine'] = production_df['Indigoidine [g/L]'].values
display(df[['Line Name','Line Description','Part ID','Media']])

,Line Name,Line Description,Part ID,Media
116,M1-R1-W1-C1-B1,"Glucose:2.8785,Na2HPO4:31.8280,NaCl:37.3632,KH...",,LB
117,M1-R2-W9-C1-B1,"Glucose:2.8785,Na2HPO4:31.8280,NaCl:37.3632,KH...",,LB
118,M1-R3-W17-C1-B1,"Glucose:2.8785,Na2HPO4:31.8280,NaCl:37.3632,KH...",,LB
119,M2-R1-W2-C1-B1,"Glucose:1.8812,Na2HPO4:29.5933,NaCl:3.7465,KH2...",,LB
120,M2-R2-W10-C1-B1,"Glucose:1.8812,Na2HPO4:29.5933,NaCl:3.7465,KH2...",,LB
121,M2-R3-W18-C1-B1,"Glucose:1.8812,Na2HPO4:29.5933,NaCl:3.7465,KH2...",,LB
122,M3-R1-W3-C1-B1,"Glucose:1.3147,Na2HPO4:47.2379,NaCl:27.7312,KH...",,LB
123,M3-R2-W11-C1-B1,"Glucose:1.3147,Na2HPO4:47.2379,NaCl:27.7312,KH...",,LB
124,M3-R3-W19-C1-B1,"Glucose:1.3147,Na2HPO4:47.2379,NaCl:27.7312,KH...",,LB
125,M4-R1-W4-C1-B1,"Glucose:0.3071,Na2HPO4:40.8515,NaCl:8.5774,KH2...",,LB


In [8]:
#Create EDD Line Description File...
df[['Line Name','Line Description','Part ID','Media']].to_csv('data/indigoidine/experiment_description.csv',index=False)

In [10]:
#Create EDD Measurement File...
df['Protocol Name'] = 'LC-MS'
df['Measurement'] = 'CID:193349:Indigoidine'
df['24'] = df['CID:193349:Indigoidine']
df[['Line Name','Protocol Name','Measurement','24']].to_csv('data/indigoidine/edd_cycle0_data.csv',index=False)